<a href="https://colab.research.google.com/github/ro-hit81/hello-world/blob/master/deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth

auth.authenticate_user()

In [0]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/vQERvBq-b7ZUxyVk_vFYv2bxGC5lwElfdEmN5VRd0lOoRm8j53Dlvx0

Successfully saved authorization token.


In [0]:
import tensorflow as tf
tf.enable_eager_execution()
print(tf.__version__)

1.15.0


In [0]:
import folium
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60']
bound = ee.FeatureCollection('users/rhtkhati/Ubon_Boundary')
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)
collection = (ee.ImageCollection("COPERNICUS/S2")
              .select(bands)
              .filter(ee.Filter.date('2019-01-01', '2019-03-31'))
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
              .map(maskS2clouds)
             )
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.TileLayer(
    tiles = mapID['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer
image = collection.sort('system:index', opt_ascending=False).mosaic()
image = image.clip(bound)
visParams = {'bands': ["B4","B3","B2"],
            'max': 0.4,
            'min': 0
                }
myMap = folium.Map(location = [15.2448, 104.8473])
myMap.add_ee_layer(image, visParams, 'sen')
myMap.add_child(folium.LayerControl())
myMap

In [0]:
labels = ee.FeatureCollection('users/rhtkhati/ubon_lulcpoint')
label = 'lulcid'
sample = image.sampleRegions(
    collection = labels, properties = [label], scale = 10).randomColumn()
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))
from pprint import pprint
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

{'training': {'geometry': None,
              'id': '00000000000000000000_0',
              'properties': {'B11': 0.23409999907016754,
                             'B12': 0.12110000103712082,
                             'B2': 0.1136000007390976,
                             'B3': 0.11020000278949738,
                             'B4': 0.10480000078678131,
                             'B8': 0.28439998626708984,
                             'QA60': 0,
                             'lulcid': 1,
                             'random': 0.43497934351769174},
              'type': 'Feature'}}
{'testing': {'geometry': None,
             'id': '00000000000000000006_0',
             'properties': {'B11': 0.2069000005722046,
                            'B12': 0.09600000083446503,
                            'B2': 0.10159999877214432,
                            'B3': 0.09560000151395798,
                            'B4': 0.07999999821186066,
                            'B8': 0.2703999876976013,
  

In [0]:
outputBucket = 'gic-rohit12.appspot.com'
print('Found Cloud Storage bucket.' if tf.gfile.Exists('gs://' + outputBucket) 
    else 'Output Cloud Storage bucket does not exist.')


Found Cloud Storage bucket.


In [0]:
trainFilePrefix = 'Training_demo'
testFilePrefix = 'Testing_demo'
featureNames = list(bands)
featureNames.append(label)

trainingTask = ee.batch.Export.table.toCloudStorage(
  collection = training,
  description ='Training Export',
  fileNamePrefix = trainFilePrefix,
  bucket = outputBucket,
  fileFormat = 'TFRecord',
  selectors = featureNames)

testingTask = ee.batch.Export.table.toCloudStorage(
  collection = testing,
  description = 'Testing Export',
  fileNamePrefix = testFilePrefix,
  bucket = outputBucket,
  fileFormat = 'TFRecord',
  selectors = featureNames)

In [0]:
trainingTask.start()
testingTask.start()

In [0]:
print(ee.batch.Task.list())
import time 
while trainingTask.active():
  print('Polling for task (id: {}).'.format(trainingTask.id))
  time.sleep(30)
print('Done with training export.')

[<Task EXPORT_FEATURES: Testing Export (READY)>, <Task EXPORT_FEATURES: Training Export (READY)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task EXPORT_FEATURES: Training Export (COMPLETED)>, <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (COMPLETED)>, <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (CANCELLED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task EXPORT_FEATURES: Training Export (COMPLETED)>, <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (CANCELLED)>, <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (FAILED)>, <Task INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/rhtkhati/Classified_pixel_demo" (FAILED)>, <Task INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/rhtkhati/Classified_pixel_demo" (FAILED)>, <Task EXPORT_IMAGE: Image Export (C

In [0]:
fileNameSuffix = '.tfrecord.gz'
trainFilePath = 'gs://' + outputBucket + '/' + trainFilePrefix + fileNameSuffix
testFilePath = 'gs://' + outputBucket + '/' + testFilePrefix + fileNameSuffix

print('Found training file.' if tf.gfile.Exists(trainFilePath) 
    else 'No training file found.')
print('Found testing file.' if tf.gfile.Exists(testFilePath) 
    else 'No testing file found.')

Found training file.
Found testing file.


In [0]:
imageFilePrefix = 'Image_pixel_demo_'

# Specify patch and file dimensions.
imageExportFormatOptions = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}

# Export imagery in this region.
exportRegion = ee.Geometry.Rectangle([104.8473, 15.24, 104.6, 15.00])

# Setup the task.
imageTask = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=imageFilePrefix,
  bucket=outputBucket,
  scale=10,
  fileFormat='TFRecord',
  region=exportRegion.toGeoJSON()['coordinates'],
  formatOptions=imageExportFormatOptions,
)

In [0]:
imageTask.start()

In [0]:
while imageTask.active():
  print('Polling for task (id: {}).'.format(imageTask.id))
  time.sleep(30)
print('Done with image export.')

Polling for task (id: O6TRWJSNNYL4ZPRPVM5UYGCP).
Polling for task (id: O6TRWJSNNYL4ZPRPVM5UYGCP).
Polling for task (id: O6TRWJSNNYL4ZPRPVM5UYGCP).
Polling for task (id: O6TRWJSNNYL4ZPRPVM5UYGCP).
Polling for task (id: O6TRWJSNNYL4ZPRPVM5UYGCP).
Polling for task (id: O6TRWJSNNYL4ZPRPVM5UYGCP).
Polling for task (id: O6TRWJSNNYL4ZPRPVM5UYGCP).
Done with image export.


In [0]:
trainDataset = tf.data.TFRecordDataset(trainFilePath, compression_type='GZIP')
print(iter(trainDataset).next())

tf.Tensor(b'\n\x88\x01\n\x0e\n\x02B2\x12\x08\x12\x06\n\x04\x1e\xa7\xe8=\n\x0e\n\x02B3\x12\x08\x12\x06\n\x04\x8a\xb0\xe1=\n\x0e\n\x02B4\x12\x08\x12\x06\n\x04b\xa1\xd6=\n\x0e\n\x02B8\x12\x08\x12\x06\n\x04\xe0\x9c\x91>\n\x0f\n\x03B11\x12\x08\x12\x06\n\x04\xe9\xb7o>\n\x0f\n\x03B12\x12\x08\x12\x06\n\x04G\x03\xf8=\n\x10\n\x04QA60\x12\x08\x12\x06\n\x04\x00\x00\x00\x00\n\x12\n\x06lulcid\x12\x08\x12\x06\n\x04\x00\x00\x80?', shape=(), dtype=string)


In [0]:
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in featureNames
]

# Dictionary with names as keys, features as values.
featuresDict = dict(zip(featureNames, columns))

pprint(featuresDict)

{'B11': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B12': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B2': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B3': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B4': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B8': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'QA60': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'lulcid': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None)}


In [0]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.
  
  Returns: 
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsedDataset = trainDataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
pprint(iter(parsedDataset).next())

({'B11': <tf.Tensor: id=54, shape=(1,), dtype=float32, numpy=array([0.2341], dtype=float32)>,
  'B12': <tf.Tensor: id=55, shape=(1,), dtype=float32, numpy=array([0.1211], dtype=float32)>,
  'B2': <tf.Tensor: id=56, shape=(1,), dtype=float32, numpy=array([0.1136], dtype=float32)>,
  'B3': <tf.Tensor: id=57, shape=(1,), dtype=float32, numpy=array([0.1102], dtype=float32)>,
  'B4': <tf.Tensor: id=58, shape=(1,), dtype=float32, numpy=array([0.1048], dtype=float32)>,
  'B8': <tf.Tensor: id=59, shape=(1,), dtype=float32, numpy=array([0.2844], dtype=float32)>,
  'QA60': <tf.Tensor: id=60, shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>},
 <tf.Tensor: id=61, shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>)


In [0]:
def normalizedDifference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).  If the denomenator is zero, add a small delta.  

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def addNDVI(features, label):
  """Add NDVI to the dataset.
  Args: 
    features: a dictionary of input tensors keyed by feature name.
    label: the target label
  
  Returns:
    A tuple of the input dictionary with an NDVI tensor added and the label.
  """
  features['NDVI'] = normalizedDifference(features['B8'], features['B4'])
  return features, label

In [0]:
from tensorflow import keras
nClasses = 5
inputDataset = parsedDataset.map(addNDVI)
def toTuple(dict, label):
  return tf.transpose(list(dict.values())), tf.one_hot(indices=label, depth=nClasses)
inputDataset = inputDataset.map(toTuple).repeat().batch(10)
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(nClasses, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=inputDataset, epochs=10, steps_per_epoch=100)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
100/100 [==============================] - 10s 102ms/step - loss: 0.9464 - acc: 0.7063
Epoch 2/10
100/100 [==============================] - 8s 79ms/step - loss: 0.4824 - acc: 0.8074
Epoch 3/10
100/100 [==============================] - 8s 85ms/step - loss: 0.4440 - acc: 0.7998
Epoch 4/10
100/100 [==============================] - 8s 81ms/step - loss: 0.4285 - acc: 0.7915
Epoch 5/10
100/100 [==============================] - 8s 81ms/step - loss: 0.4208 - acc: 0.7824
Epoch 6/10
100/100 [==============================] - 8s 80ms/step - loss: 0.4271 - acc: 0.7720
Epoch 7/10
100/100 [==============================] - 8s 79ms/step - loss: 0.4118 - acc: 0.7597
Epoch 8/10
100/100 [==============================] - 8s 79ms/step - loss: 0.4235 - acc: 0.7432
Epoch 9/10
100/100 [==============================] - 8s 78ms/step - loss: 0.4320 - acc: 0.7382
Epoch 10/10
100/100 [===================

In [0]:
testDataset = (
  tf.data.TFRecordDataset(testFilePath, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(addNDVI)
    .map(toTuple)
    .batch(1)
)

model.evaluate(testDataset, steps=45)

45/45 [==============================] - 1s 19ms/step - loss: 0.6557 - acc: 0.6889


[0.6556985729270511, 0.6888889]

In [0]:
filesList = !gsutil ls 'gs://'{outputBucket}
# Get only the files generated by the image export.
exportFilesList = [s for s in filesList if imageFilePrefix in s]

# Get the list of image files and the JSON mixer file.
imageFilesList = []
jsonFile = None
for f in exportFilesList:
  if f.endswith('.tfrecord.gz'):
    imageFilesList.append(f)
  elif f.endswith('.json'):
    jsonFile = f

# Make sure the files are in the right order.
imageFilesList.sort()

pprint(imageFilesList)
print(jsonFile)

['gs://gic-rohit12.appspot.com/Image_pixel_demo_00000.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00001.tfrecord.gz']
gs://gic-rohit12.appspot.com/Image_pixel_demo_mixer.json


In [0]:
import json

# Load the contents of the mixer file to a JSON object.
jsonText = !gsutil cat {jsonFile}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(jsonText.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 10,
 'projection': {'affine': {'doubleMatrix': [8.983152841195215e-05,
                                            0.0,
                                            104.5999215144055,
                                            0.0,
                                            -8.983152841195215e-05,
                                            15.240008626616094]},
                'crs': 'EPSG:4326'},
 'totalPatches': 100}


In [0]:
PATCH_WIDTH = mixer['patchDimensions'][0]
PATCH_HEIGHT = mixer['patchDimensions'][1]
PATCHES = mixer['totalPatches']
PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
imageColumns = [
  tf.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.float32) 
    for k in bands
]

# Parsing dictionary.
imageFeaturesDict = dict(zip(bands, imageColumns))

# Note that you can make one dataset from many files by specifying a list.
imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, imageFeaturesDict)

# Parse the data into tensors, one long tensor per patch.
imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
imageDataset = imageDataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Add additional features (NDVI).
imageDataset = imageDataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: addNDVI(features, None)[0]
)

# Turn the dictionary in each record into a tuple with a dummy label.
imageDataset = imageDataset.map(
  # Add a dummy target (-1), with a value that is obviously ridiculous.
  # This is because the model expects a tuple of (inputs, label).
  lambda dataDict: (tf.transpose(list(dataDict.values())), tf.constant(-1))
)

# Turn each patch into a batch.
imageDataset = imageDataset.batch(PATCH_WIDTH * PATCH_HEIGHT)

In [0]:
predictions = model.predict(imageDataset, steps=PATCHES, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

100/100 [==============================] - 691s 7s/step
[[0.03816325 0.55483025 0.13747299 0.14411013 0.12542345]]


In [0]:
outputImageFile = 'gs://' + outputBucket + '/Classified_pixel_demo.TFRecord'
print('Writing to file ' + outputImageFile)

Writing to file gs://gic-rohit12.appspot.com/Classified_pixel_demo.TFRecord


In [0]:
writer = tf.python_io.TFRecordWriter(outputImageFile)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], [], [], [], []]
curPatch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  patch[3].append(prediction[0][2])
  patch[4].append(prediction[0][3])
  patch[5].append(prediction[0][4])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + ' of ' + str(PATCHES) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'bareProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'vegProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
          'new1Prob': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[4])),
          'new2Prob': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[5]))
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[], [], [], [], [], []]
    curPatch += 1

writer.close()

Done with patch 1 of 100...
Done with patch 2 of 100...
Done with patch 3 of 100...
Done with patch 4 of 100...
Done with patch 5 of 100...
Done with patch 6 of 100...
Done with patch 7 of 100...
Done with patch 8 of 100...
Done with patch 9 of 100...
Done with patch 10 of 100...
Done with patch 11 of 100...
Done with patch 12 of 100...
Done with patch 13 of 100...
Done with patch 14 of 100...
Done with patch 15 of 100...
Done with patch 16 of 100...
Done with patch 17 of 100...
Done with patch 18 of 100...
Done with patch 19 of 100...
Done with patch 20 of 100...
Done with patch 21 of 100...
Done with patch 22 of 100...
Done with patch 23 of 100...
Done with patch 24 of 100...
Done with patch 25 of 100...
Done with patch 26 of 100...
Done with patch 27 of 100...
Done with patch 28 of 100...
Done with patch 29 of 100...
Done with patch 30 of 100...
Done with patch 31 of 100...
Done with patch 32 of 100...
Done with patch 33 of 100...
Done with patch 34 of 100...
Done with patch 35 of 1

In [0]:
!gsutil ls -l {outputImageFile}

 137643400  2020-01-12T08:44:18Z  gs://gic-rohit12.appspot.com/Classified_pixel_demo.TFRecord
TOTAL: 1 objects, 137643400 bytes (131.27 MiB)


In [0]:
USER_NAME = 'rhtkhati'
outputAssetID = 'users/' + USER_NAME + '/Classified_pixel_demo'
print('Writing to ' + outputAssetID)

Writing to users/rhtkhati/Classified_pixel_demo


In [0]:
!earthengine --no-use_cloud_api upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

Started upload task with ID: CLL5OHLQVLFIR5ZYROX5WYSN


In [0]:
ee.batch.Task.list()

[<Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (READY)>,
 <Task EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (COMPLETED)>,
 <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (CANCELLED)>,
 <Task EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (CANCELLED)>,
 <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (FAILED)>,
 <Task INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/rhtkhati/Classified_pixel_demo" (FAILED)>,
 <Task 

In [0]:
predictionsImage = ee.Image(outputAssetID)

predictionVis = {
  'bands': 'prediction',
  'min': 0,
  'max': 4,
  'palette': ['red', 'yellow', 'blue']
}
probabilityVis = {'bands': ['bareProb', 'vegProb', 'waterProb']}

predictionMapid = predictionsImage.getMapId(predictionVis)
probabilityMapid = predictionsImage.getMapId(probabilityVis)

map = folium.Map(location=[15.2448, 104.8473])
folium.TileLayer(
  tiles=predictionMapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='prediction',
).add_to(map)
folium.TileLayer(
  tiles=probabilityMapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map